In [14]:
import numpy as np
import plotly.graph_objects as go
import random
import scipy.stats as sps
import plotly.express as px
import pandas as pd

mu, sigma_0, delta = 0.04, 0.03, 1 
alpha, gamma = 0.5, 0.01

In [22]:
def transform_prices(beers, i, sigma):
    sigma += alpha * (sigma_0 - sigma) + gamma * sps.norm.rvs(loc=0, scale=delta)
    if beers[i] <= 6:
        beers[i] += beers[i] * (mu*delta + sigma * sps.norm.rvs(loc=0, scale=delta))
    else:
        beers[i] -= beers[i] * (mu*delta + sigma * sps.norm.rvs(loc=0, scale=delta))
    return 3.5 * beers / beers.mean(), sigma

In [23]:
def simulate(n):
    sales = [0] * 6
    total_sales = 0
    beers = np.array([3.5] * 6)
    prices = [beers]
    sigma = sigma_0
    for _ in range(n):
        i = random.randint(0, 5)
        sales[i] += 1
        total_sales += 1
        beers, sigma = transform_prices(beers, i, sigma)
        prices.append(beers)
    
    return np.array(prices), np.array(sales)

def observe_prices(prices):
    fig = go.Figure()
    for i in range(6):
        fig.add_trace(
            go.Scatter(y=prices[:, i], name=f"Bière {i+1}")
        )
    fig.update_layout(title="Evolution des prix", xaxis_title="Ventes", yaxis_title="Prix des bières")
    fig.show()

In [27]:
prices, share = simulate(n=300)
observe_prices(prices=prices)
df = pd.DataFrame({
    "Bières": [f"Bières {i}" for i in range(1, 7)],
    "Ventes": share
})
fig = px.pie(df, values='Ventes', names='Bières', title='Répartition des ventes par bières')
fig.show()